In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

/home/nlpgpu5/anaconda3/envs/kwon/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nlpgpu5/anaconda3/envs/kwon/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nlpgpu5/anaconda3/envs/kwon/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nlpgpu5/anaconda3/envs

#### Define run parameters

In [2]:
data_dir = '../data/all'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

#### Check device

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

In [5]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 2 of 38

/home/nlpgpu5/anaconda3/envs/kwon/lib/python3.6/site-packages/facenet_pytorch/models/utils/detect_face.py:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/home/nlpgpu5/anaconda3/envs/kwon/lib/python3.6/site-packages/facenet_pytorch/models/mtcnn.py:372: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  boxes = np.array(boxes)
/home/nlpgpu5/anaconda3/envs/kwon/lib/python3.6/site-packages/facenet_pytorch/models/mtcnn.py:373: VisibleDeprecationWarning: Creating an ndarray from ragged 

Batch 38 of 38

#### Define Inception Resnet V1 module

In [6]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [7]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [9]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     8/8    | loss:    3.0535 | fps:  802.2953 | acc:    0.0859   

Epoch 1/8
----------
Train |    31/31   | loss:    1.9669 | fps:  234.5460 | acc:    0.4335   
Valid |     8/8    | loss:    3.7740 | fps: 1044.8265 | acc:    0.3563   

Epoch 2/8
----------
Train |    31/31   | loss:    1.1029 | fps:  229.9842 | acc:    0.6613   
Valid |     8/8    | loss:    1.3296 | fps:  711.1356 | acc:    0.5998   

Epoch 3/8
----------
Train |    31/31   | loss:    0.7054 | fps:  233.4469 | acc:    0.7984   
Valid |     8/8    | loss:    0.8847 | fps:  815.2330 | acc:    0.7348   

Epoch 4/8
----------
Train |    31/31   | loss:    0.6002 | fps:  231.1733 | acc:    0.8075   
Valid |     8/8    | loss:    1.0622 | fps: 1042.7659 | acc:    0.7044   

Epoch 5/8
----------
Train |    31/31   | loss:    0.3965 | fps:  225.6192 | acc:    0.8700   
Valid |     8/8    | loss:    0.8874 | fps:  810.2592 | acc:    0.7066   

Epoch 6/8
----------
Train |    31/31   | loss:    0.1

#### Save fintuned model

In [10]:
torch.save(resnet.state_dict(), '../data/fintuned_model.pt')